In [ ]:
from helper import *
%matplotlib inline
# set display defaults
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels

In [ ]:
def select(imgs, n = 3, fold = 10):
    mags = [None] * (len(imgs)/2)
    for i in range(len(imgs)/2):
        mag, _ = cv2.cartToPolar(np.asarray(imgs[2*i], dtype = np.float32), np.asarray(imgs[2*i+1], dtype = np.float32))
        mags[i] = cv2.norm(mag)
    mags = np.asarray(mags)
    mags /= np.sum(mags)
    idx = []
    tmp = np.random.choice(len(mags), fold*n, replace=False, p = mags)
    idx = np.sort(tmp).reshape(-1, 3)
    return np.asarray(idx)

In [ ]:
caffe_root = '/home/bysong/caffe/'
caffe.set_mode_gpu()

model_def = './models/caffenet_siamese_cam/deploy.prototxt'
model_weights = './models/caffenet_siamese_cam/caffenet_train_iter_18174.caffemodel'
net = caffe.Net(model_def, model_weights, caffe.TEST) 
                                                                                 
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2, 0, 1))
transformer.set_mean('data', np.ones(3)*np.mean(np.load('./data/mean.npy')))

In [ ]:
fold = 5

In [ ]:
net.blobs['triple_data'].reshape(fold, 9, 227, 227)

In [ ]:
train_list, test_list = load_list('./data', dataset = 1)

In [ ]:
video = train_list[0]
print video

In [ ]:
reverse = False
flip = False

In [ ]:
if (is_forward(video) and (not reverse)) or ((not is_forward(video)) and reverse):
    direction = 'f'
else:
    direction = 'b'

In [ ]:
imgs = load_video(video, './data/ArrowDataAll/', mask = lambda x: x[:2] == 'im', grayscale = False, reverse = False, flip = flip)
flows = load_video(video, './data/ArrowDataAll/', mask = lambda x: x[:3] == 'off', grayscale = True, flip = flip, reverse = False)

In [ ]:
sel = select(flows, fold = fold)
if reverse:
    sel = sel[::-1]
images = []
for idx in sel:
    img = np.take(imgs, idx, axis = 0)
    img = map(lambda x: transformer.preprocess('data', x), img)
    img = np.asarray(np.concatenate(img, axis = 0))
    images.append(img)
images = np.asarray(images)

In [ ]:
net.blobs['triple_data'].data[...] = images

In [ ]:
sel[0]

In [ ]:
net.forward()

In [ ]:
print np.sum(net.blobs['prob'].data, axis = 0)
direction = np.sum(net.blobs['prob'].data, axis = 0).argmax()

## class activation map

### filters

In [ ]:
sz = imgs[0].shape[:2]
t = 511
plt.imshow(cv2.resize(net.blobs['conv7'].data[0][t], sz[::-1]))
plt.axis('off')
plt.tight_layout()
plt.savefig('../report/figs/cam/' + video + '-conv7-' + str(t) + '.png')

In [ ]:
img[0].shape[:2]

### CAM

In [ ]:
img = np.take(imgs, sel[0], axis = 0)
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(img[i])

In [ ]:
direction = 1

In [ ]:
sz = img[0].shape[:2]
n_figs = 1
for i in range(n_figs):
    w = net.params['fc9'][0].data[direction][512*i:512*(i+1)]
    cam = np.zeros(sz)
    for j in range(512):
        if i == 0:
            cam += w[j]*cv2.resize(net.blobs['conv7'].data[0][j], sz[::-1])
        elif i == 1:
            cam += w[j]*cv2.resize(net.blobs['conv7_p'].data[0][j], sz[::-1])
        else:
            cam += w[j]*cv2.resize(net.blobs['conv7_q'].data[0][j], sz[::-1])
    plt.subplot(1,n_figs,i+1)
    plt.imshow(cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB))
    plt.imshow(np.abs(cam), alpha = 0.5)
    plt.axis('off')
plt.tight_layout()
plt.savefig('../report/figs/cam/' + video + '-cam-' + str(n_figs) + '.png')